In [38]:
# For downloading tweets regarding #dkpol
import sys
#!{sys.executable} -m pip install searchtweets-v2

from searchtweets import ResultStream, load_credentials, collect_results, gen_request_parameters

my_search_args = load_credentials(filename="search_tweets_creds.yaml",
                 yaml_key="search_tweets_api",
                 env_overwrite=False)

In [39]:
rule = gen_request_parameters("dkpol", results_per_call=100, granularity=None) # testing with a sandbox account
print(rule)

{"query": "dkpol", "max_results": 100}


In [40]:
# creting a result stream
rs = ResultStream(request_parameters=rule,
                  max_results=100,
                  max_pages=1,
                  **my_search_args)

print(rs)
# stating that we will collect 10,000 tweets

ResultStream: 
	{
    "endpoint": "https://api.twitter.com/labs/2/tweets/search",
    "request_parameters": {
        "query": "dkpol",
        "max_results": 100
    },
    "max_tweets": 500
}


In [41]:
tweets = list(rs.stream())

 HTTP Error code: 403: {"client_id":"20542338","detail":"This request must be made using an approved developer account that is enrolled in the requested endpoint. Learn more by visiting our documentation.","registration_url":"https://developer.twitter.com/en/docs/projects/overview","title":"Client Forbidden","required_enrollment":"Standard Basic","reason":"client-not-enrolled","type":"https://api.twitter.com/labs/2/problems/client-forbidden"} | Forbidden
 Request payload: {'query': 'dkpol', 'max_results': 100}
Quitting... 


HTTPError: 

In [16]:
import pandas as pd
pd.DataFrame(tweets, columns = ["user","text"])
unfiltered_testsheet = pd.DataFrame(tweets)
unfiltered_testsheet.to_csv("Data/test_payload.csv")

In [25]:
unfiltered_testsheet['data'][0][1]

{'edit_history_tweet_ids': ['1590051348531539968'],
 'id': '1590051348531539968',
 'text': 'RT @JinoVDoabi: Irans “Parliament” har besluttet, at 14.000+ demonstranter skal henrettes. Det skal understreges, at deres kriminalitet er,…'}

In [11]:
def get_tokens(df):
    tokens = []
    for i in range(len(df)):
        tokens.append(df.iloc[i]["text"].split())
    return tokens

tokens = get_tokens(unfiltered_testsheet)
unfiltered_testsheet['text'][0]

'RT @uffeelbaek: Så kan det ikke siges mere tydeligt og klart fra FNs generalsekretær @antonioguterres: “klimasolidaritet eller kollektivt s…'

In [96]:
twitter_ids = {
    "socialdemokratiet" : {
        # "mette_frederiksen" : None,
        "magnus_heunicke" : "22695562",
        "nicolai_wammen" : "2803948786",
        "mattias_tesfaye" : "546254893",
    },
    "venstre" : {
        "jakob_ellemann" : "155584627",
        "soren_gade" : "975064362359623680",
        "sophie_lohde" : "44611200",
    },
    "moderaterne" : {
        "lars_lokke" : "26201346",
        "henrik_frandsen" : "1249019841924734977",
        "rosa_eriksen" : "1560192117858861056",
    },
    "sf" : {
        "jacob_mark" : "2373406198",
        "pia_dyhr" : "65025162",
        "kirsten_andersen" : "235646319",
    },
    "danmarksdemokraterne" : {
        # "inger_stojberg" : None,
        "dennis_flydtkjær" : "531595033",
        "peter_skaarup" : "3144074691",
        "soren_espersen" : "2444718215",
    },
    "liberal_alliance" : {
        "alex_vanopslagh" : "1531564633",
        # "henrik_dahl" : None,
        "ole_olesen" : "2222188479",
        "solbjorg_jakobsen" : "1548641745264644099",
    },
    "konservative" : {
        "soren_pape" : "2712091824",
        "mette_abildgaard" : "37877392",
        "rasmus_jarlov" : "1225930531",
    },
    "enhedslisten" : {
        "pelle_dragsted" : "119879630",
        "mai_villadsen" : "4724782641",
        "rosa_lund" : "736979161",
    },
    "radikale" : {
        "martin_lidegaard" : "1070745218",
        "samira_nawa" : "92107029",
        "katrine_robsoe" : "2491403660",
    },
    "nye_borgerlige" : {
        "pernille_vermund" : "24687777",
        "lars_mathiesen" : "980721900",
        "kim_andersen" : "783935815600799744",
    },
    "alternativet" : {
        "franciska_rosenkilde" : "777113466205274112",
        "christina_olumeko" : "1324801335372488707",
        "torsten_gejl" : "2806864609",
    },
    "dansk_folkeparti" : {
        "morten_messerschmidt" : "509288627",
        "pia_kjarsgaard" : "1054640354690039809",
        "peter_kofod" : "1613378210"
    },
}

In [107]:
import requests
import json
import pandas as pd
import os
import time

#user_id = 155584627
#search_url = "https://api.twitter.com/2/users/155584627/tweets?max_results=100&pagination_token=7140dibdnow9c7btw4232twnluqa1fddrkv35qibt8th1"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}
query_params = { #'query': keyword,
                #'start_time': start_date,
                #'end_time': end_date,
                # 'max_results': 100,
                # 'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                # 'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                # 'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                # 'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                # 'next_token': {}
                
                }

def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    #print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def main():
    for parti in twitter_ids:
        for person in twitter_ids[parti]:
            user_id = twitter_ids[parti][person]
            initial_search_url = "https://api.twitter.com/2/users/"+user_id+"/tweets?max_results=100&pagination_token=7140dibdnow9c7btw4232twnluqa1fddrkv35qibt8th1"
            json_response = connect_to_endpoint(initial_search_url, query_params)
            i = 0
            while i < 10:
                try:
                    next_token = json_response['meta']['next_token']
                except KeyError:
                    break
                temp_df = pd.DataFrame(json_response['data'])
                outdir = f'Data/{parti}'
                if not os.path.exists(outdir):
                    os.mkdir(outdir)
                temp_df.to_csv(f"{outdir}/{person}_{i}.csv")
                temp_search_url = "https://api.twitter.com/2/users/"+user_id+"/tweets?max_results=100&pagination_token=" + next_token
                print(temp_search_url)
                json_response = connect_to_endpoint(temp_search_url, query_params)
                i += 1


In [108]:
responses = main()

https://api.twitter.com/2/users/22695562/tweets?max_results=100&pagination_token=7140dibdnow9c7btw421t8ua6f3e2vzyyio89xlp3oxf9
https://api.twitter.com/2/users/22695562/tweets?max_results=100&pagination_token=7140dibdnow9c7btw420yvhd5fdv2njbhv8idvy6m3qyb
https://api.twitter.com/2/users/22695562/tweets?max_results=100&pagination_token=7140dibdnow9c7btw3z4k662x2b9xrp8rmnlq4w999jqr
https://api.twitter.com/2/users/22695562/tweets?max_results=100&pagination_token=7140dibdnow9c7btw3z3b02a9ggkcwt15k7h8rtaw4chj
https://api.twitter.com/2/users/22695562/tweets?max_results=100&pagination_token=7140dibdnow9c7btw3z21j620ttsb6zewuiyvw9228jsl
https://api.twitter.com/2/users/22695562/tweets?max_results=100&pagination_token=7140dibdnow9c7btw3z0rpm9qlwej5moybv2fvlhfm8fl
https://api.twitter.com/2/users/22695562/tweets?max_results=100&pagination_token=7140dibdnow9c7btw3w3y5czgek9mr1nqblqjg6rjyjao
https://api.twitter.com/2/users/22695562/tweets?max_results=100&pagination_token=7140dibdnow9c7btw3w33jgv062w8n

In [71]:
json.loads(responses)['meta']['next_token']
temp_df = pd.DataFrame(json.loads(responses)['data'])


,edit_history_tweet_ids,id,text
0,[1564974613137952769],1564974613137952769,"Glæder mig til søndag. Danskerne skal vide, hv..."
1,[1556342928880140288],1556342928880140288,Danskerne drømmer om at bo i ejerbolig. Men ud...
2,[1543675678238904321],1543675678238904321,Forfærdeligt angreb i Fields. Mine tanker går ...
3,[1536667473327554561],1536667473327554561,Mette Frederiksen lovede ikke at hæve ydelsern...
4,[1533448893622587392],1533448893622587392,Regeringens tvangsfordeling af gymnasieelever ...
...,...,...,...
95,[1364519751846887425],1364519751846887425,Nedlukningen har store menneskelige omkostning...
96,[1364324602109976579],1364324602109976579,Regeringen har smidt alle blå partier ud af fo...
97,[1363071780605353984],1363071780605353984,Tillykke - det er fuldt fortjent. Er glad for ...
98,[1361959774145941504],1361959774145941504,Lad os gribe genåbningen fleksibelt an. Venstr...
